In [1]:
import coremltools
from PIL import Image
import os
import re
import shutil
import json
import yaml

## Open source model

In [2]:
model_folder = '/Users/landyrev/Downloads/latest'

model_name = filter(
    lambda x: re.match(r'.*.caffemodel', x, re.IGNORECASE), 
    os.listdir(model_folder)
)[0]

model_path = os.path.join(model_folder, model_name)

## Load and convert model

In [3]:
m = coremltools.converters.caffe.convert(
    (
        model_path, os.path.join(model_folder, 'deploy.prototxt'), 
        os.path.join(model_folder, 'mean.binaryproto')
    ),
    class_labels=os.path.join(model_folder, 'labels.txt'), 
    image_input_names='data'
)

## Add model metadata

In [4]:
m.author = 'Aleksey Landyrev'
m.short_description = 'How to draw model'
m.save('realmodel.mlmodel')

## Copy model into target project

In [6]:
shutil.copy('./realmodel.mlmodel', '../../htd/Data/realmodel.mlmodel')

## Copy needed images

In [15]:
tutorials = []

shutil.rmtree('../../htd/Data/Images.bundle')
os.mkdir('../../htd/Data/Images.bundle')
labels = []

for label in open(os.path.join(model_folder, 'labels.txt')):
    label = label[:-1].replace(" ", "_")
    labels.append(label)
    if label == "random":
        continue
    
    src_dir = os.path.join('../AllImages/', label)
    dst_dir = os.path.join('../../htd/Data/Images.bundle/', label)
    meta_file = os.path.join(src_dir, 'meta.yaml')
    
    os.mkdir(dst_dir)
    
    for image in os.listdir(src_dir):
        if not re.match(r'.*(\.png|\.jpg)', image, re.IGNORECASE):
            continue
        shutil.copy(
            os.path.join(src_dir, image),
            os.path.join(dst_dir, label + "_" + image)
        )
    
    tutorial = {
        "name": label,
        "preview": os.path.join(label, label+ '_preview.png'),
        "tutorials": sorted([os.path.join(label, x) 
             for x in os.listdir(dst_dir)
             if x.endswith('.jpg')
            ])
    }
    if os.path.isfile(meta_file):
        tutorial.update(yaml.load(open(meta_file)))
    tutorials.append(tutorial)

yaml.dump(
    tutorials,
    open("../../htd/Data/Images.bundle/levels.yaml", "w")
)

In [17]:
for label in labels:
    print '"{0}": "";'.format(label)

"lutz_aquarium0": "";
"lutz_boat0": "";
"lutz_boat1": "";
"lutz_buffalo0": "";
"lutz_but1": "";
"lutz_but3": "";
"lutz_buttercup0": "";
"lutz_cat0": "";
"lutz_daisy0": "";
"lutz_dog1": "";
"lutz_duck0": "";
"lutz_el1": "";
"lutz_face2": "";
"lutz_fish0": "";
"lutz_flower0": "";
"lutz_fox0": "";
"lutz_futhsia0": "";
"lutz_globe0": "";
"lutz_goose0": "";
"lutz_heron0": "";
"lutz_horse0": "";
"lutz_jackpulit0": "";
"lutz_lighthouse0": "";
"lutz_mapple0": "";
"lutz_morningglory0": "";
"lutz_mouse1": "";
"lutz_narcissus0": "";
"lutz_owl0": "";
"lutz_owl1": "";
"lutz_peng0": "";
"lutz_pig0": "";
"lutz_plane0": "";
"lutz_plane1": "";
"lutz_plane2": "";
"lutz_pug0": "";
"lutz_rab0": "";
"lutz_rab3": "";
"lutz_radish0": "";
"lutz_snail0": "";
"lutz_squirrel0": "";
"lutz_stork0": "";
"lutz_swallow0": "";
"lutz_swallow2": "";
"lutz_swan0": "";
"lutz_turtle0": "";
"lutz_woman0": "";
"random": "";


## Try to predict test image

In [13]:
def predict(image_name):
    image = Image.open(image_name).convert('L')
    image = image.resize((224, 224))
    features = m.predict({
        'data': image
    }).get('softmax')
    for feature in features:
        print feature, int(features[feature] * 100)
        
predict('/Users/landyrev/Downloads/fvsdfvsdfv.jpg')

lutz_swan0 0
lutz_lighthouse0 0
random 0
lutz_boat0 99
lutz_pug0 0
lutz_woman0 0
